In [2]:
!pip install open-interpreter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 2.2 MB/s eta 0:00:001.8 MB/s eta 0:00:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of litellm to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of litellm to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.3/189.3 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 18.6 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 11.5 MB/s eta 0:00:0031m40.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from interpreter import OpenInterpreter

In [2]:
# agent_1 = OpenInterpreter()
# response = agent_1.chat("Testing 1 2 3")
# print(response)

Open Interpreter will require approval before running code.

Use interpreter -y to bypass this.

Press CTRL-C to exit.

We were unable to determine the context window of this model. Defaulting to 3000.

If your model can handle more, run self.context_window = {token limit}.

Also please set self.max_tokens = {max tokens per response}.

Continuing...

Output()

[{'role': 'assistant', 'type': 'message', 'content': 'Hello! How can I assist you today? If you have any specific tasks or questions, feel free to let me know!'}]


In [3]:
def print_agent_attrs(agent):
    # Get the dictionary of attributes
    attributes = agent.__dict__
    
    # Create a list of tuples containing key and type of value
    attributes_with_types = [(key, type(value).__name__) for key, value in attributes.items()]
    
    # Print the result
    for key, value_type in attributes_with_types:
        print(f"{key}: {value_type}")

In [4]:
def init_states(agent):

    agent.states = ["SPECTATE","MUTATE","DICTATE"]
    agent.state = "SPECTATE"

    return agent

In [5]:
class FIFOQueue:
    def __init__(self):
        self.queue = []

    def enqueue(self, item):
        self.queue.append(item)

    def dequeue(self):
        if not self.is_empty():
            return self.queue.pop(0)
        else:
            raise IndexError("Dequeue from an empty queue")

    def is_empty(self):
        return len(self.queue) == 0

    def size(self):
        return len(self.queue)

In [15]:
class Agent:
    def __init__(self, name, max_messages):
        self.name = name
        self.states = ["SPECTATE", "MUTATE", "DICTATE"]
        self.state = "SPECTATE"
        self.inbox = FIFOQueue()
        self.internal_state = []
        self.oi = OpenInterpreter()
        self.other_agents = []
        self.max_messages = max_messages
        self.num_messages_sent = 0
        self.generated_new_message = ""
        self.most_recently_received_message = ""

    def add_other_agent(self, agent):
        self.other_agents.append(agent)

    def handle_state(self):
        print(f"[{self.name}] Current State: {self.state}")
        print(f"[{self.name}] Inbox Size: {self.inbox.size()}")

        if self.state == "SPECTATE":
            if not self.inbox.is_empty():
                self.most_recently_received_message = self.inbox.dequeue()
                print(f"[{self.name}] Received message: {self.most_recently_received_message}")
                self.state = "MUTATE"

        elif self.state == "MUTATE":
            self.internal_state.append(self.most_recently_received_message)
            print(f"[{self.name}] Internal state after mutation: {self.internal_state}")
            self.state = "DICTATE"

        elif self.state == "DICTATE":
            if self.num_messages_sent < self.max_messages:
                messages = [{"role": "user", 
                             "content": self.most_recently_received_message["content"], 
                             "type": "message"}]
                new_message = self.oi.chat(messages, display=False, stream=False)
                print(f"[{self.name}] Generated new message: {new_message}")
                self.send_message_to_agents(self.swap_roles(new_message))
                self.num_messages_sent += 1
            else:
                print(f"[{self.name}] Max messages sent reached")
            self.most_recently_received_message = ""
            self.state = "SPECTATE"

        print(f"[{self.name}] New State: {self.state}")

    def mutate_state(self, internal_state):
        internal_state['mutated'] = True
        print(f"[{self.name}] Internal state mutated")
        return internal_state

    def send_message_to_agents(self, messages):
        for message in messages:
            for agent in self.other_agents:
                agent.inbox.enqueue(message)
                print(f"[{self.name}] Sent message to {agent.name}: {message['content']}")

    def swap_roles(self, messages):
        print("swapping roles:", messages)
        for message in messages:
            if message['role'] == 'user':
                message['role'] = 'assistant'
            elif message['role'] == 'assistant':
                message['role'] = 'user'
        print("swapped roles:", messages)
        return messages

# Create two instances of the agent with a max message limit
agent_1 = Agent("Agent 1", max_messages=5)
agent_2 = Agent("Agent 2", max_messages=5)

# Add each agent to the other's list of agents
agent_1.add_other_agent(agent_2)
agent_2.add_other_agent(agent_1)

# Simulate receiving a message
agent_1.inbox.enqueue({"role": "user", "content": "Quick! I need two cat facts! Quick!", "type": "message"})

# Run the state handler function to control the agents
agent_1.handle_state()
agent_2.handle_state()

agent_1.handle_state()
agent_2.handle_state()

agent_1.handle_state()
agent_2.handle_state()


[Agent 1] Current State: SPECTATE
[Agent 1] Inbox Size: 1
[Agent 1] Received message: {'role': 'user', 'content': 'Quick! I need two cat facts! Quick!', 'type': 'message'}
[Agent 1] New State: MUTATE
[Agent 2] Current State: SPECTATE
[Agent 2] Inbox Size: 0
[Agent 2] New State: SPECTATE
[Agent 1] Current State: MUTATE
[Agent 1] Inbox Size: 0
[Agent 1] Internal state after mutation: [{'role': 'user', 'content': 'Quick! I need two cat facts! Quick!', 'type': 'message'}]
[Agent 1] New State: DICTATE
[Agent 2] Current State: SPECTATE
[Agent 2] Inbox Size: 0
[Agent 2] New State: SPECTATE
[Agent 1] Current State: DICTATE
[Agent 1] Inbox Size: 0



We were unable to determine the context window of this model. Defaulting to 3000.

If your model can handle more, run self.context_window = {token limit}.

Also please set self.max_tokens = {max tokens per response}.

Continuing...


[Agent 1] Generated new message: [{'role': 'assistant', 'type': 'message', 'content': "I'll retrieve two cat facts for you quickly.\n\n**Plan**:\n1. Execute a code block to fetch two cat facts from an API using JavaScript.\n2. Display the cat facts to you.\n\nLet's start by fetching the cat facts."}, {'role': 'assistant', 'type': 'code', 'format': 'javascript', 'content': "const https = require('https');\n\nhttps.get('https://catfact.ninja/facts?limit=2', (resp) => {\n  let data = '';\n\n  resp.on('data', (chunk) => {\n    data += chunk;\n  });\n\n  resp.on('end', () => {\n    console.log(data);\n  });\n\n}).on('error', (err) => {\n  console.log('Error: ' + err.message);\n});"}, {'role': 'computer', 'type': 'console', 'format': 'output', 'content': ''}, {'role': 'computer', 'type': 'console', 'format': 'output', 'content': 'Output truncated. Showing the last 2800 characters.\n\nveTimeout: 0,_onPendingData: [Function: nop],agent: Agent {_events: [Object: null prototype] {free: [Functio